In [1]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs

import pyet
import pyeto
import spei

In [2]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

GEO_DATA_PATH = "./assets/geo_data"
DATABASE_PATH = "./database/database.db"

In [3]:
conn = sqlite3.connect(DATABASE_PATH)

precip_monthly = pd.read_sql(sql='SELECT * FROM precip_monthly', con=conn)
precip_monthly['Date'] = pd.to_datetime(precip_monthly['Date']) + pd.offsets.MonthEnd(0)

pet_monthly = pd.read_sql(sql='SELECT * FROM pet_monthly_modis', con=conn)
pet_monthly['Date'] = pd.to_datetime(pet_monthly['Date']) + pd.offsets.MonthEnd(0)

geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()

In [4]:
precip_monthly

,Station_ID,Date,ERA5_Precip,GPM_Precip,TRMM_Precip,TerraClimate_Precip,PERSIANNCDR_Precip,CHIRPS_Precip,PCI_ERA5,PCI_GPM,PCI_TRMM,PCI_TerraClimate,PCI_PERSIANNCDR,PCI_CHIRPS
0,40709,2000-01-31,211.450629,131.688001,128.091774,77.0,96.443996,NaN,0.832878,0.633929,0.845284,0.620690,0.708913,NaN
1,40709,2000-02-29,108.227941,77.256006,62.963703,71.0,46.755023,NaN,0.429242,0.366071,0.358505,0.484848,0.214789,NaN
2,40709,2000-03-31,89.712488,150.288005,141.844743,117.0,99.802746,NaN,0.303417,0.813084,0.957617,0.803279,0.848183,NaN
3,40709,2000-04-30,66.366945,20.880001,18.764659,18.0,24.317473,NaN,0.256723,0.037433,0.107766,0.068376,0.111865,NaN
4,40709,2000-05-31,72.468541,56.544004,51.030001,18.0,40.819520,NaN,0.188416,0.163265,0.355328,0.000000,0.113478,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7470,40735,2024-07-31,131.660313,NaN,NaN,NaN,NaN,32.320484,0.745702,NaN,NaN,NaN,NaN,0.381045
7471,40735,2024-08-31,75.881136,NaN,NaN,NaN,NaN,56.618558,0.302063,NaN,NaN,NaN,NaN,0.572843
7472,40735,2024-09-30,114.718222,NaN,NaN,NaN,NaN,174.688978,0.732031,NaN,NaN,NaN,NaN,1.000000
7473,40735,2024-10-31,79.880182,NaN,NaN,NaN,NaN,76.052103,0.244395,NaN,NaN,NaN,NaN,0.056061


In [6]:
pet_monthly

,Station_ID,Date,PET_MOD16A2GF
0,40709,1999-12-31,NaN
1,40709,2000-01-31,44.9250
2,40709,2000-02-29,72.2375
3,40709,2000-03-31,127.6375
4,40709,2000-04-30,165.7000
...,...,...,...
7220,99361,2023-08-31,239.2750
7221,99361,2023-09-30,176.6500
7222,99361,2023-10-31,123.0125
7223,99361,2023-11-30,98.8000


In [5]:
geoinfo

,Station_ID,Station_Name,Province,Station_Latitude,Station_Longitude,Station_Elevation
0,40759,Sari,Mazandaran,36.536,52.998,23.0
1,99306,Bandar-e-amirabad,Mazandaran,36.856,53.386,-20.0
2,99357,Baladeh,Mazandaran,36.198,51.801,2120.0
3,99299,Galugah,Mazandaran,36.738,53.837,-10.0
4,40737,Gharakhil,Mazandaran,36.487,52.108,14.7
5,40760,Kiyasar,Mazandaran,36.248,53.546,1294.3
6,99361,Alasht,Mazandaran,36.071,52.843,1805.0
7,99309,Amol,Mazandaran,36.479,52.468,23.7
8,99348,Kojur,Mazandaran,36.390,51.729,1550.0
9,99360,Polsefid,Mazandaran,36.104,53.062,610.0


In [7]:
data_si = precip_monthly\
    .merge(pet_monthly, on=['Station_ID', 'Date'], how='outer')\
    .set_index(['Date'])[['Station_ID', 'ERA5_Precip', 'GPM_Precip', 'PET_MOD16A2GF']]

data_si['PE_ERA5'] = data_si['ERA5_Precip'] - data_si['PET_MOD16A2GF']
data_si['PE_GPM'] = data_si['GPM_Precip'] - data_si['PET_MOD16A2GF']


timescale = [1, 3, 6, 9, 12, 15, 18, 21, 24]
timescale = [3]
i = 1

for ts in timescale:
    df_spi = data_si\
        .groupby(by='Station_ID')\
        .apply(
            lambda x: spei.spi(
                series=x.GPM_Precip,
                dist=scs.gamma,
                prob_zero=True,
                timescale=ts
            ),
            include_groups=False
        )\
        .reset_index()\
        .rename(columns={0: f'SPI_{ts}'})
    if i == 1:
        results = df_spi.copy()
    else:
        results = results.merge(df_spi, on=['Station_ID', 'Date'], how='outer')
    
    df_spei = data_si\
        .groupby(by='Station_ID')\
        .apply(
            lambda x: spei.spei(
                series=x.PE_GPM,
                dist=scs.fisk,
                prob_zero=True,
                timescale=ts
            ),
            include_groups=False
        )\
        .reset_index()\
        .rename(columns={0: f'SPEI_{ts}'})
    results = results.merge(df_spei, on=['Station_ID', 'Date'], how='outer')

    i += 1


results

KeyError: 'Date'

In [8]:
results

Date,Station_ID,2000-03-31 00:00:00,2000-04-30 00:00:00,2000-05-31 00:00:00,2000-06-30 00:00:00,2000-07-31 00:00:00,2000-08-31 00:00:00,2000-09-30 00:00:00,2000-10-31 00:00:00,2000-11-30 00:00:00,...,2023-09-30 00:00:00,2023-10-31 00:00:00,2023-11-30 00:00:00,2023-12-31 00:00:00,2024-01-31 00:00:00,2024-02-29 00:00:00,2024-03-31 00:00:00,2024-04-30 00:00:00,2024-05-31 00:00:00,2024-06-30 00:00:00
0,40709,1.885282,0.238665,0.046399,-1.372798,-0.716743,-0.181585,1.813167,1.752298,1.423438,...,0.787786,-0.202432,-1.217805,-0.887044,0.891372,2.194951,1.302340,-0.819044,1.636674,1.957534
1,40718,0.880326,-0.193711,-0.343143,-1.064547,-0.280822,-0.120594,0.319835,1.181815,1.225447,...,0.990663,-0.430212,-1.325916,-1.641705,-0.737617,0.727787,0.742691,-0.578949,0.375077,1.482202
2,40719,0.723483,-0.217809,-0.379278,-1.072976,-0.581762,-0.191817,0.208804,1.112361,1.088017,...,1.029075,-0.374006,-1.094640,-1.422300,-0.743205,0.450138,0.429755,-0.565427,0.447659,1.470886
3,40720,0.459525,-0.319015,-0.536630,-1.078742,-1.552212,-0.384032,0.229640,1.174055,1.068117,...,0.736005,-0.687978,-1.361739,-1.773103,-1.551681,-0.118488,0.160355,-0.705911,0.573839,1.385009
4,40732,1.073653,0.049812,-0.695439,-0.692005,-0.946990,-0.437023,-0.284424,0.633251,0.372496,...,0.751497,-0.621189,-0.886057,-0.925611,0.017551,1.272106,0.281665,0.035274,0.230779,1.213336
5,40734,0.574069,-0.045824,-1.429065,-1.542575,-0.770241,-0.409558,-0.149060,0.960270,0.822516,...,0.390030,-0.072660,-0.158598,-0.151660,0.736721,1.827078,0.656593,-0.036335,0.233693,2.094458
6,40735,0.283525,-1.133394,-1.723681,-1.682069,-1.059669,-0.541240,-0.234212,1.028716,0.882645,...,0.295574,0.040033,-0.359983,-0.473922,-0.082985,1.419052,0.474354,-0.478292,0.062607,1.726938
7,40736,0.835921,-0.058975,-1.278352,-0.919281,-0.036390,0.108456,0.449781,1.080459,0.939133,...,0.027300,0.410358,0.003274,-0.385087,-0.797233,0.669680,0.282792,0.377189,-0.304140,1.691159
8,40737,0.655600,-0.723643,-1.455298,-1.296946,-0.353941,-0.065600,0.356180,1.216519,1.242514,...,-0.002833,0.268061,0.082370,-0.217700,-0.501255,0.532662,0.000338,-0.082596,-0.162776,2.004359
9,40759,0.757204,-0.026754,-1.245819,-0.936608,0.045516,0.203870,0.451344,1.075372,0.860921,...,-0.074148,0.323899,0.053551,-0.360144,-0.823135,0.247407,-0.098948,0.267458,-0.083898,1.718628


In [7]:
conn = sqlite3.connect(DATABASE_PATH)

results.to_sql('si_monthly', conn, if_exists='replace', index=False)

conn.commit()
conn.close()